<h1> Approximating a Region of the Sine Function </h1>

In this notebook we will approximate a region of the sine function with a neural network to get a sense of how architecture and hyperparameters affect neural network performance. We will be introducing Pytorch Neural Network structure, Pytorch datasets and dataloaders and optimizers

<img src="../data/sine_wave.gif" width="1200" align="center">


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

from tqdm.notebook import trange, tqdm

<h2> Pytorch Datasets and Dataloaders </h2>
<b> *Basics only, we will cover more detail later on, but for now...</b><br>
Pytorch has a huge number of functionalities that make training our neural networks very easy! One of those functionalities is the Pytorch dataset and dataloader (they are real life-savers!). The "dataset" class is an object that "stores" our dataset either directly (it loads all the data in the initialisation function) or indirectly (it loads the image paths during the initialisation function and only loads them when it needs to - for large image-based datasets this is usually the only way to do it).We will see how we can create our own Pytorch dataset soon!<br>
These datasets then are used to create a "dataloader" object that is "iterable". The Pytorch dataloader will take our dataset and randomly shuffle it (if we tell it to), it will also divide the dataset into "mini-batches" which are groups of datapoints of a fixed size (the batch size). Our Neural Network is then trained through a single step of GD on this mini-batch. As we iterate through the dataloader, the dataloader will pass us a new unique mini-batch until the whole dataset has been passed to us. One whole loop through the dataset is called an "epoch", during every epoch the dataset is re-shuffled so the mini-batches are all random. This random sampling of the dataset and training on mini-batches (instead of performing GD on the whole dataset) is called Stochastic Gradient Descent (SGD)<br>
Note: If the whole dataset does not evenly divide into mini batches then in the last iterator we will just be passed whatever is left over!

<h3> Creating a Pytorch dataset </h3>
The dataset we will be creating will be points from a "noisey" sine wave.<br>
The Pytorch dataset class has three essential parts:<br>
The __init__ function (as most Python classes do)<br>
The __getitem__ function (this is called during every iteration)<br>
The __len__ function (this must return the length of the dataset)

In [ ]:
# Create a "SineDataset" class by importing the Pytorch Dataset class
class SineDataset(Dataset):
    """ Data noisey sinewave dataset
        num_datapoints - the number of datapoints you want
    """
    def __init__(self, num_datapoints):
        # Lets generate the noisy sinewave points
        
        # Create "num_datapoints" worth of random x points using a uniform distribution (0-1) using torch.rand
        # Then scale and shift the points to be between -9 and 9
        self.x_data = torch.rand(num_datapoints,1)*18 - 9 
        
        # Calculate the sin of all data points in the x vector and the scale amplitude
        self.y_data = (torch.sin(self.x_data))/2.5
        
        # Add some gaussein noise to each datapoint using torch.randn_like
        # Note:torch.randn_like will generate a tensor of gaussein noise the same size 
        # and type as the provided tensor
        self.y_data += torch.randn_like(self.y_data)/20

    def __getitem__(self, index):
        # This function is called by the dataLOADER class whenever it wants a new mini-batch
        # The dataLOADER class will pass the dataSET and number of datapoint indexes (mini-batch of indexes)
        # It is up to the dataSET's __getitem__ function to output the corresponding input datapoints 
        #AND the corresponding labels
        return self.x_data[index], self.y_data[index]
        # Note:Pytorch will actually pass the __getitem__ function one index at a time
        # If you use multiple dataLOADER "workers" multiple __getitem__ calls will be made in parallel
        # (Pytorch will spawn multiple threads)

    def __len__(self):
        # We also need to specify a "length" function, Python will use this fuction whenever
        # You use the Python len(function)
        # We need to define it so the dataLOADER knows how big the dataSET is!
        return self.x_data.shape[0]

Now that we've defined our dataset, lets create an instance of it for training and testing and then create  dataloaders to make it easy to iterate

In [ ]:
n_x_train = 30000   # the number of training datapoints
n_x_test = 8000     # the number of testing datapoints
batch_size = 16

# Create an instance of the SineDataset for both the training and test set
dataset_train = SineDataset(n_x_train)
dataset_test  = SineDataset(n_x_test)

# Now we need to pass the dataSET to the Pytorch dataLOADER class along with some other arguments
# batch_size - the size of our mini-batches
# shuffle - whether or not we want to shuffle the dataset
data_loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
data_loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

Lets visualise the dataset we've created!!

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.scatter(dataset_train.x_data, dataset_train.y_data, s=0.2)
# Note:see here how we can just directly access the data from the dataset class

<h2> Neural Network Architecture</h2>
<b> Non-Linear function approximators! </b> <br>

Up until now we have only created a single linear layer with an input layer and an output layer. In this section we will start to create multi-layered networks with many "hidden" layers separated by "activation functions" that give our networks "non-linearities". If we didn't have these activation functions and simply stacked layers together, our network would be no better than a single linear layer! Why? Because multiple sequential "linear transformations" can be modeled with just a single linear transformation. This is easiest to understand with matrix multiplications (which is exactly what happens inside a linear layer).<br>

$M_o = M_i*M_1*M_2*M_3*M_4*M_5$<br>
Is the same as<br>
$M_o = M_i*M_T$<br>
Where<br>
$M_T = M_1*M_2*M_3*M_4*M_5$<br>

Aka multiplication with several matrices can be simplified to multiplication with a single matrix.<br>

So what are these nonlinear activation functions that turn our simple linear models into a power "nonlinear function approximator"? Some common examples are:<br>
1. relu
2. sigmoid
3. tanh

Simply put they are "nonlinear" functions, the simplest of which is the "rectified linear unit" (relu) which is "piecewise non-linear".

NOTE: The term "layer" most commonly refers to the inputs or outputs of the weight matrix or activations functions and not the linear layer or activation layer themselves. Output layers in between two "linear layers" are called "hidden layers". You can imagine them "inside" the neural network with us only being able to see the input and output layers. To confuse things even further the outputs of activation functions are also commonly called "activations"

Why do we want a linear function approximator? Because many processes, tasks, systems in the real world are non-linear. "Linear" in basic terms refers to any process that takes inputs, scales them and sums them together to get an output. 

<h3>Pytorch nn.Module</h3>
Now we can define a Pytorch model to be trained!<br>
To do so we use the Pytorch nn.Module class as the base for defining our network. Just like the dataset class, this class has a number of important functions.

In [ ]:
# Define our network class by using the nn.module
class ShallowLinear(nn.Module):
    '''
    A simple, general purpose, fully connected network
    '''
    # Here we initialise our network and define all the layers we need
    def __init__(self, input_size, output_size, hidden_size):
        # Perform initialization of the pytorch superclass, this will allow us to inherit 
        # functions from the nn.Module class
        super(ShallowLinear, self).__init__()
        # Define the linear layers we will need
        # Note: the output of one layer must be the same size 
        # as the input to the next!
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.linear4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # This function is an important one and we must create it or pytorch will give us an error!
        # This function defines the "forward pass" of our neural network
        # and will be called when we simply call our network class
        # aka we can do net(input) instead of net.forward(input)
        
        # Lets define the sqeuence of events for our forward pass!
        x = self.linear1(x) # hidden layer
        x = torch.tanh(x)   # activation function
        
        x = self.linear2(x) # hidden layer
        x = torch.tanh(x)   # activation function
        
        x = self.linear3(x) # hidden layer
        x = torch.tanh(x)   # activation function

        # No activation function on the output!!
        x = self.linear4(x) # output layer
        
        # Note we re-use the variable x as we don't care about overwriting it 
        # though in later labs we will want to use earlier hidden layers
        # later in our network!
        return x


## Define hyperparameters,  model and optimizer

Here we define the following parameters for training:

- batch size (which has already been defined)
- learning rate
- number of training epochs
- optimizer
- loss function

Ideally, numeric parameters would be tested empirically with an exhaustive search. When testing manually, It is recommended to maximize the model fit with one parameter at a time to avoid confounding your results. 

Try these learning rates:
- 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5

Try these optimizers:
- `optim.SGD(shallow_model.parameters(), lr=learning_rate)`
- `optim.Adam(shallow_model.parameters(), lr=learning_rate)`

[Youtube: Optimizers - EXPLAINED! by CodeEmporium](https://youtu.be/mdKjMPmcWjY?si=VtjuF_QlPHAzj2Sx)

See the pytorch documentation pages for an extensive list of options:
- Optimizers: http://pytorch.org/docs/master/optim.html#algorithms
- Loss: http://pytorch.org/docs/master/nn.html#id46


In [ ]:
# Define the hyperparameters
learning_rate = 5e-4
nepochs = 10

# Create model
shallow_model = ShallowLinear(input_size=1, output_size=1, hidden_size=64)

# Initialize the optimizer with above parameters
optimizer = optim.Adam(shallow_model.parameters(), lr=learning_rate)

# Define the loss function
loss_fn = nn.MSELoss()  # mean squared error

## Initiate training, plot testing results
Here we put all the previous methods together to train and test the model. This problem is an unusual one in that our loss is the best quantitative metric of the model performance. Classification problems require further analysis of true/false positives/negatives.

Rerun this cell several times without editing any parameters. Is the result the same?

Try a larger batch size, how is the training time affected?

Look at the slope and noise level of the loss plot. Does it look like the training converged on a local minimum?

Try some different hyperparameters and see how accurate you can get your model

In [ ]:
# Here we create two lists to store the loss values from training and testing
training_loss_logger = []
testing_loss_logger = []
# Note: create them outside of the train/test cell so they don't get overwritten 
# if we want to run the cell again

In [ ]:
# The main train/test cell
# This will run one epoch of training and then one epoch of testing nepochs times

for epoch in trange(nepochs, desc="Epochs", leave=False):
    
    # Perform training Loop!
    for x, y in tqdm(data_loader_train, desc="Training", leave=False):

        # Run forward calculation
        y_predict = shallow_model(x)

        # Compute loss.
        loss = loss_fn(y_predict, y)

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable weights
        # of the model)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()

        # Log the loss so we can visualise the training plot later
        training_loss_logger.append(loss.item())

    # Perform a test Loop!
    with torch.no_grad():
        test_loss_accum = 0
        for i, (x, y) in enumerate(tqdm(data_loader_test, desc="Testing", leave=False)):

            # Run forward calculation
            y_predict = shallow_model(x)
            # Compute loss.
            loss = loss_fn(y_predict, y)
            # Log the loss so we can visualise the training plot later
            testing_loss_logger.append(loss.item())
            test_loss_accum += loss
            
        test_loss_accum /= (i + 1)
        
print("Epoch [%d/%d], Average Test Loss %.4f" %(epoch, nepochs, test_loss_accum))

<h3>Lets visualise our results!</h3>

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(np.linspace(0, nepochs, len(training_loss_logger)), training_loss_logger)
plt.plot(np.linspace(0, nepochs, len(testing_loss_logger)), testing_loss_logger)
title = plt.title("Training and Testing Loss Vs Epoch")
legend = plt.legend(["Training", "Testing"])

In [ ]:
test_outputs = []
# Perform a test Loop!
with torch.no_grad():
    for x, y in data_loader_test:
        # Run forward calculation
        y_predict = shallow_model(x)
        test_outputs.append(y_predict.detach().cpu().numpy())

In [ ]:
# Plot the testdata against the model's predicted outputs, how accurate is it?
fig = plt.figure(figsize=(20, 10))
plt.scatter(dataset_test.x_data, dataset_test.y_data, s=0.2)
plt.scatter(dataset_test.x_data, test_outputs, s=0.2)
plt.text(-9, 0.44, "- Prediction", color="orange", fontsize=16)
plt.text(-9, 0.48, "- Sine wave (with noise)", color="blue", fontsize=16)